In [ ]:
# Import
import os
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
from matplotlib.axes import Axes
from matplotlib.figure import Figure
import matplotlib.font_manager as font_manager
import matplotlib.patches as mpatches
from info import *

# Import local_features and util (hack)
import sys
import os
sys.path.append(os.path.abspath(".."))
from local_features import *
from util import *


INFO = Info()


In [ ]:
# Set Heuristica as font

font_path = os.path.join(os.path.abspath('..'), 'font.otf')
assert os.path.exists(font_path)
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

plt.rcParams['font.family'] = prop.get_name()
plt.rcParams['font.size'] = 12

In [ ]:
# Piano roll for a specified performance
from recording import OutputName

participant_id: int = 0
performance_id: int = 2

performance: Performance = INFO.data(participant_id, performance_id)
print(ALGORITHM_LONG_NAMES[performance.algorithm])
recording: Recording = performance.recording
midi_notes: list[MidiNote] = all_notes(recording.measures)

fig: Figure
ax: Axes
fig, ax = plt.subplots(figsize=(20, 5))

# Plot each note with the corresponding color
for note in midi_notes:
    color = "red" if note.output_name == OutputName.LOOPBACK else "blue"
    ax.broken_barh([(note.time, note.length)], (note.note - 0.4, 0.8), facecolors=color)

# Formatting
ax.invert_yaxis()
ax.set_xticks(np.arange(0, 48+4, 4))
ax.set_yticks([])

ax.grid(True)

# Add legend
legend_patches = [
    plt.Line2D([0], [0], color="red", lw=4, label="Pupil"),
    plt.Line2D([0], [0], color="blue", lw=4, label="Algorithm")
]
ax.legend(handles=legend_patches, loc="upper right")

plt.show()

In [ ]:
# Ratings per algorithm (side-to-side bars)

fig, axs = plt.subplots(NUM_QUESTIONS, figsize=(8, 10))

colors = ['#B45C1F', '#DE8B52', '#1FB42A', "#52DE5C"]

questions = [
    "I feel that the algorithm could respond well to me",
    "I feel that I could respond well to the algorithm",
    "I feel that the algortihm gives me inspiration",
    "I enjoyed playing this song with the algorithm",
]

width = 0.166

for algorithm_index in range(NUM_ALGORITHMS):
    # Get scores of performances of algorithm
    performances = [
        INFO.info[id]
        for id in INFO.info
        if INFO.info[id].algorithm == algorithm_index 
    ]

    print(performances[0].scores)

    for q in range(NUM_QUESTIONS):
        q_scores: list[int] = [sum([p.scores[p.participant_id][q] == v for p in performances]) for v in range(1, 6)]
        axs[q].title.set_text(f'"{questions[q]}"')
        axs[q].bar(x=np.array(range(1, 6)) + width * algorithm_index - width*1.5,height=q_scores, width=width, color=colors[algorithm_index])
        axs[q].set_yticks(range(0, 10, 5))

legend_patches = [
    mpatches.Patch(color=colors[algorithm_index], label=ALGORITHM_SHORT_NAMES[algorithm_index]) for algorithm_index in range(NUM_ALGORITHMS)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/response_scores.png')

In [ ]:
# Metrics over fours (dot plot)

fig, axs = plt.subplots(NUM_ALGORITHMS, figsize=(8, 10))

colors = ['#ADB220', '#1F77B4']

width = 0.166

for algorithm_index in range(NUM_ALGORITHMS):
    # Get performances of algorithm
    performances = [
        INFO.info[id]
        for id in INFO.info
        if INFO.info[id].algorithm == algorithm_index 
    ]

    metrics = make_average_list(48 // 4)

    # Get metrics of all fours
    for performance in performances:
        for i, four in enumerate(performance.recording.fours):
            if i >= (48 // 4): continue
            metrics[i].add(interval_avg(four))

    avg_metrics = get_average(metrics)

    x = list(range(len(metrics)))
    c = [colors[i % 2] for i in x]

    ax = axs[algorithm_index]

    ax.scatter(x, avg_metrics, c=c, marker='o')
    ax.set_yticks(range(7))

legend_patches = [
    mpatches.Patch(color=colors[i], label=["Human four", "Algorithm four"][i]) for i in range(2)
]
fig.legend(handles=legend_patches, loc="outside upper right")

plt.savefig('charts/response_scores.png')

In [ ]:
# Edit distances per algorithm

colors = ['#B45C1F', '#DE8B52', '#1FB42A', "#52DE5C"]

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.set_title('Token edit distances per algorithm')

all_averages = [0] * NUM_ALGORITHMS

for algorithm_index in range(NUM_ALGORITHMS):
    # Get performances of algorithm
    performances = [
        INFO.info[id]
        for id in INFO.info
        if INFO.info[id].algorithm == algorithm_index 
    ]

    # Get average edit score for all performances of this algorithm
    avg = Avg()
    for performance in performances:
        avg.add(np.average(performance.edit_distances))
    average_edit_distance = avg.avg()

    all_averages[algorithm_index] = average_edit_distance


ax.set_ylabel('Edit distance')
ax.bar(ALGORITHM_SHORT_NAMES, all_averages, color=colors)

plt.show()

In [ ]:
# Self / peer correlation

self_scores = []
peer_scores = []

for id in INFO.info:
    performance = INFO.info[id]

    p_self_scores = performance.scores[performance.participant_id]

    others = [performance.scores[par] for par in range(NUM_PARTICIPANTS) if par != performance.participant_id]
    p_peer_scores = [float(np.average(z)) for z in zip(others[0], others[1])]

    self_scores += p_self_scores
    peer_scores += p_peer_scores

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.set_title('Scatter')

ax.scatter(self_scores, peer_scores)